In [0]:
import os
from dotenv import load_dotenv
from google import genai
from google.genai import types
import json
from datetime import datetime 

In [0]:
# Carrega as variaveis de ambiente
load_dotenv()

api_llm = os.getenv("GEMINI_API_KEY")
DEFAULT_PATH = os.getenv("DEFAULT_PATH")

current_date = datetime.today().strftime('%Y-%m-%d')

In [0]:
# lendo os dados da tabela silver
df = spark.read.parquet(f"{DEFAULT_PATH}/Silver/*.parquet")

# Transforma os dados em uma estrutura json para melhor entendimento do LLM
dados_json = df.toPandas().to_json()

In [0]:
prompt_analise = f"""
**FUNÇÃO:** Você é um **Analista de Câmbio Sênior** com foco no Real Brasileiro (BRL). Sua tarefa é processar os dados de moedas fornecidos no formato JSON e gerar uma análise completa com insights.

**DADOS DE ENTRADA (JSON):**
[{dados_json}]

**INSTRUÇÕES DE ANÁLISE:**

1.  **Moedas Extremas:** Identifique as **3 moedas mais fortes** e as **3 mais fracas** em relação ao BRL (com base no valor mais alto e mais baixo da 'diferenca_brl').
2.  **Média Continental:** Calcule a **média aritmética** da 'diferenca_brl' para cada continente listado.
3.  **Insights:** Gere **3 insights** profissionais, claros e acionáveis, explicando o que cada um significa para um investidor brasileiro.

**FORMATO DE SAÍDA OBRIGATÓRIO:**

A resposta deve ser **SOMENTE** o texto da análise, formatado em **Markdown** (com títulos, listas, e **negrito**), totalmente em **Português do Brasil**. Não inclua o JSON de entrada na saída e não adicione nenhum comentário antes do início da análise.
"""

In [0]:
# Envia o prompt para o LLM
try:
    # Conexão com o LLM
    # obs: ele localiza a api_key no arquivo .env com o nome de GEMINI_API_KEY
    client = genai.Client()

    # Envia o prompt para o LLM
    resposta = client.models.generate_content( 
        model='gemini-2.5-flash', 
        contents=prompt_analise
    )

    resultado = resposta.text

    print("--- Insights Gerados ---")
    print(resultado)
    print("------------------------")

    if resultado != None:
        try:
            with open(f'{DEFAULT_PATH}/Gold/relatorio_llm_{current_date}.txt', 'w') as txt:
                txt.write(resultado)
        except Exception as erro:
            print(f"Não foi possível criar o arquivo de relatorio pelo seguinte erro: {erro}")
except Exception as erro:
    print('Não foi possível se conectar ao LLM')
    print(erro)


